# Heart Disease Prediction

## Data Exploration
This notebook focuses on the initial exploration of the dataset to understand its structure, data types, and potential issues. The goal is to perform a general analysis, unify the representation of missing values, and ensure that all columns have consistent and appropriate data types. Additionally, the cleaned dataset will be stored in a parquet format for further processing.

`Simón Correa Marín`

### **1. Import Libraries**

In [ ]:
# base libraries for data science
from pathlib import Path
import numpy as np
import pandas as pd
import pyarrow as pa

: 